In [ ]:
%time
import time
start = time.time()

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.0-preview2/spark-3.0.0-preview2-bin-hadoop3.2.tgz
!tar xf spark-3.0.0-preview2-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-preview2-bin-hadoop3.2"

In [ ]:
%time 
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.streaming import StreamingContext
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType

In [ ]:
#Spark Contexto, Session , SQLContexto
%time 
ss = SparkSession.builder.master("local[*]").getOrCreate()
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
sql_sc = SQLContext(sc)
ssc = StreamingContext(sc, 2)

In [ ]:
%time 
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
! wget -c s3.amazonaws.com/data-sprints-eng-test/data-sample_data-nyctaxi-trips-2012-json_corrigido.json

In [ ]:
%time
import time
start = time.time()

pandas_df2012 = pd.read_json('/content/data-sample_data-nyctaxi-trips-2012-json_corrigido.json', lines=True)

print('seconds: {}'.format(time.time()-start))

In [ ]:
%time
import time
start = time.time()

s_df2012 = sql_sc.createDataFrame(pandas_df2012)
s_df2012.columns
s_df2012.printSchema()
s_df2012.show()

print('seconds: {}'.format(time.time()-start))

In [ ]:
%time
import time
start = time.time()

s_df2012.createOrReplaceTempView("trips2012")

#sqlDF = sql_sc.sql("SELECT DATE_FORMAT(pickup_datetime, 'd/m/Y') AS data, count(tip_amount) AS total FROM trips2012 WHERE tip_amount > 0 GROUP BY pickup_datetime ORDER BY pickup_datetime ASC")
sqlDF = sql_sc.sql("SELECT DATE_FORMAT(pickup_datetime, 'Y/m/d') AS dia, count(tip_amount) AS total FROM trips2012 WHERE tip_amount > 0 GROUP BY pickup_datetime ORDER BY pickup_datetime ASC")
sqlDF.show()

print('seconds: {}'.format(time.time()-start))

In [ ]:
%time 
sqlDF.repartition(1).write.csv('/content/resultado2.csv', header=True)

In [ ]:
%time
df2012N = pd.read_csv('/content/resultado2.csv/part-00000-eed87269-345d-4663-8dfb-06d811c78ad4-c000.csv')
df2012N

In [ ]:
%time
s_dfN = sql_sc.createDataFrame(df2012N)
s_dfN.show()

In [ ]:
s_dfN = df2012N.plot.line(x='dia', y='total', figsize=(10, 10))

In [ ]:
s_dfN = pd.DataFrame({'total'},{'dia'})
s_dfN = df2012N.plot.line()